In [53]:
from myutils import *
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度

In [54]:
pickle_file = '/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/dr_20240417_1452001e84_anchor_by_event/dl_lost/results/BR_mle_20240417_1636000_b9ecf_iter2.pkl'
with open(pickle_file, 'rb') as f:
    res_records = pickle.load(f)

def find_matching_element(data, criteria):
    for item in data:
        file_path = item[7]
        if all(criterion in file_path for criterion in criteria):
            return item
    return None

res_dirpath = '/home/wmnlab/F/results/20240417_1333007d66_new_data_sync_v2/dr_20240417_1452001e84_anchor_by_event/dl_lost/BR_20240417_1636000_b9ecf_iter2'

In [55]:
selected_dates = data_loader(query_dates=True, show_info=False)
selected_dates = ["2024-03-19"]
filepaths = data_loader('dr', selected_dates=selected_dates, selected_bands=['B1B8+B3B8', 'B3B7+B3B8'])
pprint(filepaths)

[([('sm04', 'B1B8', '#01', '2024-03-19', 'UDP_Bandlock_9S_Phone_BR', 'BR'),
   '/home/wmnlab/F/database/2024-03-19/UDP_Bandlock_9S_Phone_BR/sm04/#01/data/handover_info_log.csv',
   '/home/wmnlab/F/database/2024-03-19/UDP_Bandlock_9S_Phone_BR/sm04/#01/data/udp_dnlk_loss_latency.csv',
   '/home/wmnlab/F/database/2024-03-19/UDP_Bandlock_9S_Phone_BR/sm04/#01/data/udp_uplk_loss_latency.csv',
   '/home/wmnlab/F/database/2024-03-19/UDP_Bandlock_9S_Phone_BR/sm04/#01/data/diag_log_sm04_2024-03-19_18-39-12_rrc.csv',
   '/home/wmnlab/F/database/2024-03-19/UDP_Bandlock_9S_Phone_BR/sm04/#01/data/diag_log_sm04_2024-03-19_18-39-12_ml1.csv',
   '/home/wmnlab/F/database/2024-03-19/UDP_Bandlock_9S_Phone_BR/sm04/#01/data/diag_log_sm04_2024-03-19_18-39-12_nr_ml1.csv'],
  [('sm06', 'B3B8', '#01', '2024-03-19', 'UDP_Bandlock_9S_Phone_BR', 'BR'),
   '/home/wmnlab/F/database/2024-03-19/UDP_Bandlock_9S_Phone_BR/sm06/#01/data/handover_info_log.csv',
   '/home/wmnlab/F/database/2024-03-19/UDP_Bandlock_9S_Phone_B

In [56]:
for i, filepath in enumerate(filepaths):
    # if i > 0:
    #     break
    
    # pprint(filepath)
    print(filepath[0][0])
    print(filepath[1][0])
    
    eutra_df1, eutra_df2 = set_data(gen_dataframe(filepath[0][5]), 'eutra'), set_data(gen_dataframe(filepath[1][5]), 'eutra')
    
    eutra_df1['Band'] = eutra_df1['EARFCN'].apply(lambda x: earfcn2band(x))
    eutra_df1 = eutra_df1[eutra_df1['RSRP'] != -30.0]
    eutra_df2['Band'] = eutra_df2['EARFCN'].apply(lambda x: earfcn2band(x))
    eutra_df2 = eutra_df2[eutra_df2['RSRP'] != -30.0]
    
    pcell_df1 = eutra_df1[eutra_df1['serv_cel_index'] == 'PCell'].copy().reset_index(drop=True)
    pcell_df2 = eutra_df2[eutra_df2['serv_cel_index'] == 'PCell'].copy().reset_index(drop=True)
    
    res_path = os.path.join(res_dirpath, filepath[0][2].replace('/', '\\').replace('.csv', '')+filepath[1][2].replace('/', '\\'))
    data = gen_dataframe(res_path, parse_dates=['Timestamp_x', 'Timestamp_y'], usecols=['Timestamp_x', 'Timestamp_y', 'anchor_type', 'type_x', 'type_y', 'lost_x', 'lost_y', 'lost', 'Y_x_0', 'Y_y_0', 'Y_0', 'Y_x_1', 'Y_y_1', 'Y_1'])
    # data['Y'] = data[['Y_0', 'Y_1', 'Y_2', 'Y_3', 'Y_4']].sum(axis=1) >= 2
    
    # Merge data with pcell_df1
    merged_data = pd.merge_asof(data, pcell_df1[['Timestamp', 'Band']], left_on='Timestamp_x', right_on='Timestamp', direction='backward', suffixes=('', '_x'))
    merged_data.rename(columns={'Band': 'Band_x'}, inplace=True)
    del merged_data['Timestamp']
    # display(merged_data)

    # Merge merged_data with pcell_df2
    final_merged_data = pd.merge_asof(merged_data, pcell_df2[['Timestamp', 'Band']], left_on='Timestamp_y', right_on='Timestamp', direction='backward', suffixes=('', '_y'))
    final_merged_data.rename(columns={'Band': 'Band_y'}, inplace=True)
    del final_merged_data['Timestamp']
    # display(final_merged_data)

    # Count rows with NaN in 'Band_x' or 'Band_y'
    initial_row_count = final_merged_data.shape[0]
    final_merged_data = final_merged_data.dropna(subset=['Band_x', 'Band_y']).reset_index(drop=True)
    final_row_count = final_merged_data.shape[0]

    # Calculate and print the number of rows removed
    na_rows_count = initial_row_count - final_row_count
    print(f"Rows removed due to NaN in 'Band_x' or 'Band_y': {na_rows_count}")

    # Optionally, display the DataFrame to verify
    # display(final_merged_data)
    
    df = final_merged_data.copy()

    # print(df)
    print(round(df['lost'].mean() * 100, 3))

    df['Time_diff'] = df['Timestamp_x'].diff().fillna(pd.Timedelta(seconds=0))  # Calculate difference and handle the first row
    
    # # Replace the NaN in the first row's Time_diff with 0 (as timedelta)
    df.at[0, 'Time_diff'] = pd.Timedelta(seconds=0)

    # Group by Band_x and Band_y pairs
    total_times = df.groupby(['Band_x', 'Band_y'])['Time_diff'].sum()
    
    # # Ensure all bands are included
    # bands = ['B1', 'B3', 'B7', 'B8']
    # total_times = total_times.reindex(bands, fill_value=pd.Timedelta(seconds=0))

    # Convert Timedelta to total seconds for easier calculation and display
    total_times_seconds = total_times.dt.total_seconds()

    # Create a DataFrame from the grouped data
    result_df = pd.DataFrame(total_times_seconds, columns=['Total Duration']).reset_index()
    result_df['Total Duration (sec)'] = total_times_seconds  # Include seconds explicitly

    # Calculate total duration in minutes
    result_df['Total Duration (min)'] = (result_df['Total Duration (sec)'] / 60).map("{:.2f} min".format)

    # Calculate the total duration in seconds for percentage calculation
    total_duration_seconds = result_df['Total Duration (sec)'].sum()

    # Calculate the percentage of total duration for each band pair
    result_df['Total Duration (%)'] = (result_df['Total Duration (sec)'] / total_duration_seconds * 100).map("{:.2f}%".format)

    del result_df['index'], result_df['Total Duration']
    result_df = result_df.reset_index()

    # Display the final DataFrame
    display(result_df)

    # Calculate time differences between consecutive rows
    df['Time_diff'] = df['Timestamp_x'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds()

    # Group by Band and Type pairs
    grouped = df.groupby(['Band_x', 'Band_y', 'type_x', 'type_y'])

    # Calculate aggregates
    aggregated = grouped.agg(
        count=('Time_diff', 'size'),  # Number of rows in each group
        lost=('lost', 'sum'),  # Sum of the 'lost' column
        lost_rate=('lost', 'mean'),  # Mean of the 'lost' column
        y0=('Y_0', 'sum'),  # Sum of the 'Y_0' column
        y0_rate=('Y_0', 'mean'),  # Mean of the 'Y_0' column
        y1=('Y_1', 'sum'),  # Sum of the 'Y_1' column
        y1_rate=('Y_1', 'mean'),  # Mean of the 'Y_1' column
        total_duration=('Time_diff', 'sum')  # Total duration in seconds
    )

    # Convert total duration from seconds to minutes
    aggregated['total_duration (min)'] = aggregated['total_duration'] / 60

    # Calculate the percentage of total duration for each group
    total_duration_seconds = aggregated['total_duration'].sum()
    aggregated['total_duration (%)'] = (aggregated['total_duration'] / total_duration_seconds) * 100
    
    aggregated[['lost_rate', 'y0_rate', 'y1_rate']] = (aggregated[['lost_rate', 'y0_rate', 'y1_rate']] * 100).round(3)
    aggregated[['total_duration', 'total_duration (min)', 'total_duration (%)']] = aggregated[['total_duration', 'total_duration (min)', 'total_duration (%)']].round(2)

    # Reset index to make Band_x, Band_y, type_x, and type_y regular columns
    aggregated.reset_index(inplace=True)
    
    sorted_aggregated = aggregated.sort_values(
        by=['Band_x', 'Band_y', 'lost', 'lost_rate'], 
        ascending=[True, True, False, False]
    ).reset_index(drop=True)
    # sorted_aggregated = sorted_aggregated.set_index(['Band_x', 'Band_y'])

    # Display the final DataFrame
    display(sorted_aggregated)

('sm04', 'B1B8', '#01', '2024-03-19', 'UDP_Bandlock_9S_Phone_BR', 'BR')
('sm06', 'B3B8', '#01', '2024-03-19', 'UDP_Bandlock_9S_Phone_BR', 'BR')
Rows removed due to NaN in 'Band_x' or 'Band_y': 2733
0.067


,Band_x,Band_y,Total Duration (sec),Total Duration (min),Total Duration (%)
0,B1,B3,125.003819,2.08 min,4.48%
1,B8,B3,2375.671033,39.59 min,85.11%
2,B8,B8,290.489573,4.84 min,10.41%


,Band_x,Band_y,type_x,type_y,count,lost,lost_rate,y0,y0_rate,y1,y1_rate,total_duration,total_duration (min),total_duration (%)
0,B1,B3,MCGF,SCGM,1327,76,5.727,38,2.864,33,2.487,2.65,0.04,0.10
1,B1,B3,ENBH,MCGF,47,0,0.000,0,0.000,0,0.000,0.09,0.00,0.00
2,B1,B3,ENBH,MNBH,913,0,0.000,0,0.000,0,0.000,1.83,0.03,0.07
3,B1,B3,ENBH,SCGM,4162,0,0.000,0,0.000,0,0.000,8.32,0.14,0.30
4,B1,B3,ENBH,Stable,1797,0,0.000,0,0.000,0,0.000,3.59,0.06,0.13
5,B1,B3,LTEH,MCGH,482,0,0.000,0,0.000,0,0.000,0.96,0.02,0.03
6,B1,B3,LTEH,MNBH,1136,0,0.000,0,0.000,0,0.000,2.27,0.04,0.08
7,B1,B3,LTEH,SCGC-II,650,0,0.000,0,0.000,0,0.000,1.30,0.02,0.05
8,B1,B3,LTEH,SCGM,3173,0,0.000,0,0.000,0,0.000,6.35,0.11,0.23
9,B1,B3,LTEH,Stable,8456,0,0.000,0,0.000,4,0.047,16.91,0.28,0.61


('sm05', 'B3B7', '#01', '2024-03-19', 'UDP_Bandlock_9S_Phone_BR', 'BR')
('sm06', 'B3B8', '#01', '2024-03-19', 'UDP_Bandlock_9S_Phone_BR', 'BR')
Rows removed due to NaN in 'Band_x' or 'Band_y': 2854
0.539


,Band_x,Band_y,Total Duration (sec),Total Duration (min),Total Duration (%)
0,B3,B3,2024.132585,33.74 min,72.53%
1,B3,B8,27.187269,0.45 min,0.97%
2,B7,B3,476.042243,7.93 min,17.06%
3,B7,B8,263.302304,4.39 min,9.44%


...

,Band_x,Band_y,Total Duration (sec),Total Duration (min),Total Duration (%)
0,B1,B3,159.813457,2.66 min,7.08%
1,B1,B8,0.668030,0.01 min,0.03%
2,B8,B3,1780.075191,29.67 min,78.89%
3,B8,B8,315.894761,5.26 min,14.00%


,Band_x,Band_y,type_x,type_y,count,lost,lost_rate,y0,y0_rate,y1,y1_rate,total_duration,total_duration (min),total_duration (%)
0,B1,B3,ENBH,MCGF,943,0,0.000,0,0.000,0,0.000,1.89,0.03,0.08
1,B1,B3,ENBH,MCGH,22,0,0.000,0,0.000,0,0.000,0.04,0.00,0.00
2,B1,B3,ENBH,MNBH,591,0,0.000,0,0.000,0,0.000,1.18,0.02,0.05
3,B1,B3,ENBH,SCGC-II,196,0,0.000,0,0.000,0,0.000,0.39,0.01,0.02
4,B1,B3,ENBH,SCGM,4200,0,0.000,0,0.000,0,0.000,8.40,0.14,0.37
5,B1,B3,ENBH,SCGR-II,286,0,0.000,0,0.000,0,0.000,0.57,0.01,0.03
6,B1,B3,ENBH,Stable,3410,0,0.000,0,0.000,0,0.000,6.82,0.11,0.30
7,B1,B3,LTEH,ENBH,97,0,0.000,0,0.000,0,0.000,0.19,0.00,0.01
8,B1,B3,LTEH,LTEH,26,0,0.000,0,0.000,0,0.000,0.05,0.00,0.00
9,B1,B3,LTEH,MCGF,664,0,0.000,0,0.000,0,0.000,1.33,0.02,0.06


('sm05', 'B3B7', '#02', '2024-03-19', 'UDP_Bandlock_9S_Phone_BR', 'BR')
('sm06', 'B3B8', '#02', '2024-03-19', 'UDP_Bandlock_9S_Phone_BR', 'BR')
Rows removed due to NaN in 'Band_x' or 'Band_y': 3323
0.255


,Band_x,Band_y,Total Duration (sec),Total Duration (min),Total Duration (%)
0,B3,B3,1792.039738,29.87 min,64.57%
1,B3,B8,0.778036,0.01 min,0.03%
2,B7,B3,666.735156,11.11 min,24.02%
3,B7,B8,315.784755,5.26 min,11.38%


,Band_x,Band_y,type_x,type_y,count,lost,lost_rate,y0,y0_rate,y1,y1_rate,total_duration,total_duration (min),total_duration (%)
0,B3,B3,MCGF,MCGF,17973,3054,16.992,2768,15.401,2729,15.184,35.95,0.60,1.30
1,B3,B3,MCGH,MCGF,2935,311,10.596,113,3.850,0,0.000,5.87,0.10,0.21
2,B3,B3,SCGC-II,MCGF,903,84,9.302,8,0.886,0,0.000,1.81,0.03,0.07
3,B3,B3,SCGM,SCGM,117444,42,0.036,16,0.014,28,0.024,234.90,3.91,8.46
4,B3,B3,MCGF,SCGM,11010,13,0.118,31,0.282,46,0.418,22.02,0.37,0.79
5,B3,B3,SCGM,MCGH,4282,3,0.070,0,0.000,0,0.000,8.56,0.14,0.31
6,B3,B3,SCGM,SCGC-II,5680,1,0.018,0,0.000,0,0.000,11.36,0.19,0.41
7,B3,B3,ENBH,ENBH,340,0,0.000,0,0.000,0,0.000,0.68,0.01,0.02
8,B3,B3,ENBH,SCGA,1300,0,0.000,0,0.000,0,0.000,2.60,0.04,0.09
9,B3,B3,ENBH,Stable,2707,0,0.000,0,0.000,2,0.074,5.41,0.09,0.20
